In [1]:
!export CUDA_VISIBLE_DEVICES=1

In [2]:
import argparse
import os
import math
import random
import json
import numpy as np

from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

from encoder import Encoder
from attention import Attention
from decoder import Decoder
from generator import Generator

from data_loader import SpeechDataset, Padding, ToTensor

Using TensorFlow backend.


In [3]:
torch.cuda.set_device(1)

In [4]:
#args = parser.parse_args()

In [5]:
with open('labels_dict.json', 'r') as f:
    labels = json.loads(f.read())
    
len(labels)

32

In [6]:
SIGNAL_INPUT_SIZE = 1500 
TXT_INPUT_SIZE = 135
OUTPUT_DIM = 33
BATCH_SIZE = 32

audio_conf = {'window': 'hamming',
              'window_size' : 0.02,
              'window_stride' : 0.01,
              'sampling_rate': 16000}

train_dataset = SpeechDataset('../../SpeechRecognition.EN/deepspeech.cv/data/cv-valid-dev_manifest.csv', 
                            'labels_dict.json',
                            audio_conf,
                            transform=transforms.Compose([Padding(SIGNAL_INPUT_SIZE, TXT_INPUT_SIZE, 'labels_dict.json')]) 
                              )

val_dataset = SpeechDataset('../../SpeechRecognition.EN/deepspeech.cv/data/cv-other-dev_manifest.csv', 
                            'labels_dict.json',
                            audio_conf,
                            transform=transforms.Compose([Padding(SIGNAL_INPUT_SIZE, TXT_INPUT_SIZE, 'labels_dict.json')]) 
                              )

train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE,
                            shuffle=True, num_workers=4)
val_dataloader = DataLoader(val_dataset, batch_size=BATCH_SIZE,
                            shuffle=True, num_workers=4)

In [7]:
NUM_GRU = 6
ENC_HID_DIM = 256
DEC_HID_DIM = 256 
DEC_EMB_DIM = 256
DROPOUT_RATE = 0.2

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#device = torch.device('cpu')

encoder = Encoder(SIGNAL_INPUT_SIZE, NUM_GRU, ENC_HID_DIM, DEC_HID_DIM, DROPOUT_RATE, device)
attention = Attention(enc_hid_dim=ENC_HID_DIM, dec_hid_dim=DEC_HID_DIM)
decoder = Decoder(OUTPUT_DIM, DEC_EMB_DIM, ENC_HID_DIM, DEC_HID_DIM, DROPOUT_RATE, attention)
model = Generator(encoder, decoder, device).to(device)

In [8]:
#model

In [9]:
if model.cuda:
    print(True)
else:
    print(False)

True


In [10]:
optimizer = optim.Adam(model.parameters())
pad_idx = labels['pad']

criterion = nn.CrossEntropyLoss(ignore_index=pad_idx)

In [11]:
def train(model, iterator, optimizer, criterion, clip):
    
    model.train()
    
    epoch_loss = 0
    
    for i, sample in enumerate(iterator):
        
        src = sample['signal'].type(torch.FloatTensor).to(device)
        trg = sample['transcript'].type(torch.LongTensor).to(device)
        trg = trg.view(-1, TXT_INPUT_SIZE)
       
        optimizer.zero_grad()
        
        #print('src.size - ', src.size(), ' trg.size - ', trg.size())
        #print(trg[:, 0])
        _, _, output = model(src, trg)
        #print('src.size - ', src.size(), ' trg.size - ', trg.size(), ' output.size - ', output.size())
        
        loss = criterion(output.view(-1, output.shape[2]), trg.view(-1))
        
        loss.backward()
        
        #torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        
        optimizer.step()
        
        epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [12]:
def evaluate(model, iterator, criterion):
    
    model.eval()
    
    epoch_loss = 0
    #print('in evaluation')
    
    with torch.no_grad():
    
        for i, sample in enumerate(iterator):

            src = sample['signal'].type(torch.FloatTensor).to(device)
            trg = sample['transcript'].type(torch.LongTensor).to(device)
            trg = trg.view(-1, TXT_INPUT_SIZE)

            _, _, output = model(src, trg, 0) #turn off teacher forcing
            #print('output - ', output)
            
            loss = criterion(output.view(-1, output.shape[2]), trg.view(-1))

            epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [13]:
N_EPOCHS = 25
CLIP = 10
SAVE_DIR = 'models'
MODEL_SAVE_PATH = os.path.join(SAVE_DIR, 'rsr_gan.pt')

best_valid_loss = float('inf')

if not os.path.isdir(SAVE_DIR):
    os.makedirs(SAVE_DIR)

for epoch in range(N_EPOCHS):
    with torch.cuda.device(1):
        train_loss = train(model, train_dataloader, optimizer, criterion, CLIP)
        valid_loss = evaluate(model, val_dataloader, criterion)

        if valid_loss < best_valid_loss:
            best_valid_loss = valid_loss
            torch.save(model.state_dict(), MODEL_SAVE_PATH)
            print('Saved Model.')

        print('Epoch: ', epoch+1, '| Train Loss: ', train_loss, '| Train PPL: ', math.exp(train_loss),
              '| Val. Loss: ', valid_loss, '| Val. PPL: ', math.exp(valid_loss))
    
    #print('Train loss - ', train_loss)

TypeError: clip_grad_norm_() missing 1 required positional argument: 'max_norm'

In [ ]:
torch.cuda.device(1)